本节内容我们将讲述如何使用AutoGen从0开始构建一个智能体，内容包括环境的搭建、智能体的构建以及标准内容输出。

### 环境安装

参考我们在[Anaconda](https://xxx)介绍的关于Anaconda的安装方法，在现有Anaconda环境的基础上，我们搭建一个运行Autogen的基础Python环境，取名为`wtf_agents`。

#### 创建虚拟环境

1. AutoGen要求 Python 3.10 或更高版本，因此我们创建一个 Python 3.12 的虚拟环境。

```bash
conda create -n wtf_agents python=3.12 -y
```

2. 创建虚拟环境成功后，我们激活该虚拟环境。
```bash
conda activate wtf_agents
```

3. 安装AutoGen的依赖包。

```bash
pip install autogen
```